In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import numpy as np

In [2]:
plt.ion()

## Loading the data

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
# Training dataset
train_loader = torch.utils.data.DataLoader(
                datasets.MNIST(root='', train=True, download=True,
                               transform = transforms.Compose([
                                   transforms.ToTensor(),
                                   transforms.Normalize((0.1307,), (0.3081,))
                               ])),
                               batch_size = 64,
                               shuffle = True,
                               num_workers = 4
                              )
                

# Test dataset
test_loader = torch.utils.data.DataLoader(
                datasets.MNIST(root='', train=False, 
                               transform = transforms.Compose([
                                   transforms.ToTensor(),
                                   transforms.Normalize((0.1307,), (0.3081,))
                               ])),
                               batch_size = 64,
                               shuffle = True,
                               num_workers = 4
                              )

0it [00:00, ?it/s]

9920512it [00:04, 2338566.87it/s]                             


Extracting MNIST/raw/train-images-idx3-ubyte.gz to MNIST/raw


0it [00:00, ?it/s]

32768it [00:00, 38483.99it/s]                           
0it [00:00, ?it/s]

Extracting MNIST/raw/train-labels-idx1-ubyte.gz to MNIST/raw


1654784it [00:00, 1956087.84it/s]                            
0it [00:00, ?it/s]

Extracting MNIST/raw/t10k-images-idx3-ubyte.gz to MNIST/raw


8192it [00:00, 16176.58it/s]            

Extracting MNIST/raw/t10k-labels-idx1-ubyte.gz to MNIST/raw
Processing...
Done!


## Define Network

In [15]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        # Define network elements
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)
        
        # Define spatial transformer localization-network
        self.localization = nn.Sequential(
                                nn.Conv2d(1, 8, kernel_size=7),
                                nn.MaxPool2d(2, stride=2),
                                nn.ReLU(True),
                                nn.Conv2d(8, 10, kernel_size=5),
                                nn.MaxPool2d(2, stride=2),
                                nn.ReLU(True)
        )
        
        # Regressor for the 3 * 2 affine network
        self.fc_loc = nn.Sequential(
                        nn.Linear(10*3*3, 32),
                        nn.ReLU(True),
                        nn.Linear(32, 3*2)
        )
        
        # Initialize the weights / bias with indentity transformation
        self.fc_loc[2].weight.data.zero_()
        self.fc_loc[2].bias.data.copy_(torch.tensor([1, 0, 0, 0, 1, 0], dtype=torch.float))
        
        # Spatial transformer network forward function
        # 데이터를 localization하고 affine grid를 구하는 과정
    def stn(self, x):
        xs = self.localization(x)
        xs = xs.view(-1, 10*3*3)
        theta = self.fc_loc(xs)
        theta = theta.view(-1, 2, 3)

        grid = F.affine_grid(theta, x.size())
        x = F.grid_sample(x, grid)

        return x
                
    def forward(self, x):
        # 우선 Spatial Transformer에 태워서 정해진 위치를 강조한 데이터를 얻음
        x = self.stn(x)
        
        # 그 데이터에 우리가 보통 태우는 CNN을 태운다
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)
        
        
model = Net().to(device)

## Train model with SGD

In [23]:
optimizer = optim.SGD(model.parameters(), lr=0.01)

def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        
        if batch_idx % 500 == 0:
            print("Train Epoch : {} [{}/{} ({:.0f})]\tLoss: {:.6f}".format(
                    epoch, batch_idx * len(data), len(train_loader.dataset), 100. * batch_idx / len(train_loader), loss.item()))
            
            
def test():
    with torch.no_grad():
        model.eval()
        test_loss = 0
        correct = 0
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            
            test_loss += F.nll_loss(output, target, size_average=False).item()
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()
            
        test_loss /= len(test_loader.dataset)
        print("\nTest Set : Average loss : {:.4f}, Accuracy: {}/{} ({:.0f}%)\n".format(
        test_loss, correct, len(test_loader.dataset), 100. * correct / len(test_loader.dataset)))

In [24]:
for epoch in range(1, 10+1):
    train(epoch)
    test()

Train Epoch : 1 [0/60000 (0)]	Loss: 0.193307
Train Epoch : 1 [32000/60000 (53)]	Loss: 0.210227

Test Set : Average loss : 0.0945, Accuracy: 9733/10000 (97%)

Train Epoch : 2 [0/60000 (0)]	Loss: 0.341377
Train Epoch : 2 [32000/60000 (53)]	Loss: 0.085150

Test Set : Average loss : 0.0739, Accuracy: 9764/10000 (98%)

Train Epoch : 3 [0/60000 (0)]	Loss: 0.116831
Train Epoch : 3 [32000/60000 (53)]	Loss: 0.289644

Test Set : Average loss : 0.0717, Accuracy: 9794/10000 (98%)

Train Epoch : 4 [0/60000 (0)]	Loss: 0.171285
Train Epoch : 4 [32000/60000 (53)]	Loss: 0.098717

Test Set : Average loss : 0.0563, Accuracy: 9823/10000 (98%)

Train Epoch : 5 [0/60000 (0)]	Loss: 0.084532
Train Epoch : 5 [32000/60000 (53)]	Loss: 0.211168

Test Set : Average loss : 0.0589, Accuracy: 9821/10000 (98%)

Train Epoch : 6 [0/60000 (0)]	Loss: 0.258731
Train Epoch : 6 [32000/60000 (53)]	Loss: 0.168707

Test Set : Average loss : 0.1026, Accuracy: 9671/10000 (97%)

Train Epoch : 7 [0/60000 (0)]	Loss: 0.098769
Train E